In [1]:
import numpy as np
import pandas as pd
import datetime

import matplotlib
import matplotlib.pyplot as plt

In [2]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
from sklearn import metrics
from sklearn.metrics import fbeta_score, make_scorer

In [3]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [4]:
link_df = pd.read_csv('../../data/original/links (table 3).csv', index_col=0)

In [5]:
validation_path = '../../data/online/'

In [6]:
morning_gbdt_regressor_dict = {}
for link_id in link_df.index.values:
    print(link_id)
    X_train_df = pd.read_csv(validation_path + 'feature/feature7/morning/X_train_link' + str(link_id) + '.csv', index_col = 0)
    y_train_df = pd.read_csv(validation_path + 'feature/feature7/morning/y_train_link' + str(link_id) + '.csv', index_col = 0)
    morning_gbdt_regressor_dict[link_id] = []
    
    X_train, X_validation, y_train, y_validation = train_test_split(X_train_df.values, y_train_df.values, random_state=7, test_size=0.5)
    gbdt_regressor = GradientBoostingRegressor(n_estimators=10)
    gbdt_regressor.fit(X_train, y_train)
    predict = gbdt_regressor.predict(X_validation)
    #print(' mape=%f'%Mape(predict, y_validation))
    morning_gbdt_regressor_dict[link_id] = gbdt_regressor

100
101
102
103


C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123


In [7]:
afternoon_gbdt_regressor_dict = {}
for link_id in link_df.index.values:

    X_train_df = pd.read_csv(validation_path + 'feature/feature7/afternoon/X_train_link' + str(link_id) + '.csv', index_col = 0)
    y_train_df = pd.read_csv(validation_path + 'feature/feature7/afternoon/y_train_link' + str(link_id) + '.csv', index_col = 0)
    afternoon_gbdt_regressor_dict[link_id] = []
    
    X_train, X_validation, y_train, y_validation = train_test_split(X_train_df.values, y_train_df.values, random_state=7, test_size=0.5)
    gbdt_regressor = GradientBoostingRegressor(n_estimators=10)
    gbdt_regressor.fit(X_train, y_train)
    predict = gbdt_regressor.predict(X_validation)
    #print(' mape=%f'%Mape(predict, y_validation))
    afternoon_gbdt_regressor_dict[link_id] = gbdt_regressor
        

C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Test

In [8]:
morning_predict_link_arrive_time_dict = {}
for link_id in link_df.index.values:

    X_test_df = pd.read_csv(validation_path + 'feature/feature7/morning/X_test_link' + str(link_id) + '.csv', index_col = 0)
    
    morning_predict_link_arrive_time_dict[link_id] = []
    for d in range(len(X_test_df)):
        gbdt_regressor = morning_gbdt_regressor_dict[link_id]
        predict = gbdt_regressor.predict(X_test_df.loc[d].values.reshape(1, -1))
        morning_predict_link_arrive_time_dict[link_id].append(predict)

In [9]:
afternoon_predict_link_arrive_time_dict = {}
for link_id in link_df.index.values:

    X_test_df = pd.read_csv(validation_path + 'feature/feature7/afternoon/X_test_link' + str(link_id) + '.csv', index_col = 0)
    afternoon_predict_link_arrive_time_dict[link_id] = []
    for d in range(len(X_test_df)):

        gbdt_regressor = afternoon_gbdt_regressor_dict[link_id]
        predict = gbdt_regressor.predict(X_test_df.loc[d].values.reshape(1, -1))
            
        afternoon_predict_link_arrive_time_dict[link_id].append(predict)

In [10]:
for link_id in link_df.index.values:
    predict_df = pd.DataFrame(afternoon_predict_link_arrive_time_dict[link_id])
    predict_df.to_csv(validation_path + 'prediction/feature7/afternoon/' + str(link_id) + '.csv')
    
    predict_df = pd.DataFrame(morning_predict_link_arrive_time_dict[link_id])
    predict_df.to_csv(validation_path + 'prediction/feature7/morning/' + str(link_id) + '.csv')